In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

import acbm
from acbm.validating.plots import plot_comparison, plot_activity_sequence_comparison, plot_intrazonal_trips
from acbm.validating.utils import calculate_od_distances, process_sequences


## 1. Overview figure of AcBM

TODO: add some description
![acbm](https://urban-analytics-technology-platform.github.io/blog_content/v2_release/acbm_diagram.png)


## 2. Read generated activity chains


The four outputs from the AcBM pipline are the following files:
- `people.csv` (SPC)
- `households.csv` (SPC)
- TODO: add SPC builder to load people and households
- `activities.csv` (AcBM)
- `legs.csv` (AcBM (primary), PAM (secondary))
- `legs_with_locations.parquet` (AcBM (primary), PAM (secondary))
- `plans.xml` (AcBM, PAM): MATSim output include background demographics as extra features

In [ ]:
processed_dir = "data/processed/activities_pam/"

# People
people = pd.read_csv(acbm.root_path / processed_dir / "people.csv")


In [ ]:
people.head()

In [ ]:
# Households
housholds = pd.read_csv(acbm.root_path / processed_dir / "households.csv")
housholds.head()

In [ ]:
# Activities: (rows are trips wit times, duration and zone (of the activity) and ordinal value for trip within the day)
activities = pd.read_csv(acbm.root_path / processed_dir / "activities.csv")[[
    "pid", "hid", "activity", "seq", "start time", "end time", "duration", "zone"
]]
activities

## Activities

__Rows are activities in a specific location with times, duration and zone (of the activity) and ordinal value for trip within the day__

In [ ]:

activities_ = pd.read_csv(acbm.root_path / processed_dir / "activities.csv")[[
    "pid", "hid", "activity", "seq", "start time", "end time", "duration", "zone"
]]
activities

### Legs with locations

__Each row is a trip linking activities in a given day for a given person__

Notes (TODO: add some more descriptions for noteworthy columns):
- location IDs match with the osmox POI geoparquet file downloaded in script `0.1_osmox.py`

In [ ]:
# TODO: Pick two HIDs as examples (person 6 and person 18) if need to have fast runtime

legs_with_locations = pd.read_parquet(
    acbm.root_path / "data/processed/activities_pam/legs_with_locations.parquet"
)
legs_with_locations[
    [
        "pid",
        "hid",
        "ozone",
        "dzone",
        "origin activity",
        "destination activity",
        "mode",
        "seq",
        "tst",
        "tet",
        "duration",
        "start_location_id",
        "end_location_id",
        "start_location_geometry_wkt",
        "end_location_geometry_wkt"
    ]
]

# Add geometries
legs_with_locations["start_location_geometry"] = gpd.GeoSeries.from_wkt(legs_with_locations["start_location_geometry_wkt"])
legs_with_locations["end_location_geometry"] = gpd.GeoSeries.from_wkt(legs_with_locations["end_location_geometry_wkt"])



In [ ]:
from shapely import LineString

legs_with_locations["geometry"] = legs_with_locations[
    ["start_location_geometry", "end_location_geometry"]
].apply(
    lambda row: LineString(
        [row["start_location_geometry"], row["end_location_geometry"]]
    )
    if row["start_location_geometry"] is not None
    and row["end_location_geometry"] is not None
    else pd.NA,
    axis=1,
)


In [ ]:
# Load boundaries
boundaries = gpd.read_file(
    acbm.root_path / "data/external/boundaries" / "oa_england.geojson",
    where="MSOA21NM LIKE '%Leeds%'",
).to_crs("epsg:4326")


In [ ]:
geo_legs_with_locations = gpd.GeoDataFrame(legs_with_locations, crs="epsg:4326")

In [ ]:
# generate subset and distinct dataframes from start, end, trip
hids = [3, 7]
pids = [6, 18]
# hids = range(1, 100)
# pids = range(1, 100)

legs_subset = geo_legs_with_locations[geo_legs_with_locations["hid"].isin(hids)]
start = legs_subset[["pid", "hid", "start_location_geometry", "origin activity", "seq"]]
end = legs_subset[["pid", "hid", "end_location_geometry", "destination activity"]]
trips = legs_subset[["pid", "hid", "geometry", "mode", "seq"]]

modes = trips["mode"].unique()  # Collect all unique modes
colormap = plt.colormaps.get_cmap("Dark2")  # Generate a colormap
mode_colors = {mode: colormap(i) for i, mode in enumerate(modes)}  # Map modes to colors
activities_unique = start["origin activity"].unique()  # Collect all unique modes
colormap = plt.colormaps.get_cmap("Accent")  # Generate a colormap
act_colors = {act: colormap(i) for i, act in enumerate(activities_unique)}  # Map modes to colors

In [ ]:
activities_unique

In [ ]:
# TODO: next steps
# - add legend
# - add time as title of subplot
# - explore other trips
# - try animation (see e.g. https://matplotlib.org/stable/api/animation_api.html)
# - label nodes (work, home, etc)
# - timestamp/seq on nodes, duration text on edges
# - add arrows for linestrings (e.g. see https://stackoverflow.com/a/77558203)

import matplotlib.pyplot as plt

for hid in start["hid"].unique():
    for pid in start[start["hid"].eq(hid)]["pid"].unique():
        print(hid, pid)
        start_p = start[start["pid"].eq(pid)]
        end_p = end[end["pid"].eq(pid)]
        trips_p = trips[trips["pid"].eq(pid)]

        fig, ax = plt.subplots(1, trips_p["seq"].unique().shape[0])
        fig.set_size_inches(15, 10)
        for i, seq in enumerate(trips_p["seq"]):
            boundaries.plot(ax=ax[i], color="lightgrey")
            start_p.set_geometry("start_location_geometry").plot(ax=ax[i], color=start_p["origin activity"].map(act_colors))
            end_p.set_geometry("end_location_geometry").plot(ax=ax[i], color=end_p["destination activity"].map(act_colors))

            if trips_p[trips_p["seq"].lt(seq)].shape[0] > 0:
                trips_p[trips_p["seq"].lt(seq)].plot(ax=ax[i], color=trips_p["mode"].map(mode_colors), alpha=0.1)
            trips_p[trips_p["seq"].eq(seq)].plot(ax=ax[i], color=trips_p["mode"].map(mode_colors), alpha=1.0)
            # print(start_p)
            # print(end_p)
            # print(trips_p)
            ax[i].set_title("title", fontsize="medium")
            ax[i].axis("off")
        plt.show()

# Read MATSim XML

Output of AcBM is converted to MATSim XML format using PAM for downstream travel demand or agent based modelling. An example of the output from AcBM for two people is shown below:

```xml
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE population SYSTEM "http://matsim.org/files/dtd/population_v6.dtd">

<population>

<!-- ====================================================================== -->

<person id="6">
  <attributes>
    <attribute class="java.lang.String" name="hzone">E00059032</attribute>
    <attribute class="java.lang.Integer" name="age_years">82</attribute>
    <attribute class="java.lang.Integer" name="hid">3</attribute>
  </attributes>
  <plan selected="yes">
    <activity type="home" start_time="00:00:00" end_time="10:47:13" x="-1.3876467539043311" y="53.932630464963154"/>
    <leg mode="car" trav_time="00:30:00"/>
    <activity type="other" start_time="11:17:13" end_time="12:45:33" x="-1.538353198422133" y="53.79629354711603"/>
    <leg mode="car" trav_time="00:20:00"/>
    <activity type="home" start_time="13:05:33" end_time="14:42:43" x="-1.3876467539043311" y="53.932630464963154"/>
    <leg mode="car" trav_time="00:30:00"/>
    <activity type="escort" start_time="15:12:43" end_time="15:31:17" x="-1.563287162580116" y="53.799973062659674"/>
    <leg mode="car" trav_time="00:20:00"/>
    <activity type="home" start_time="15:51:17" end_time="17:51:05" x="-1.3876467539043311" y="53.932630464963154"/>
    <leg mode="car" trav_time="00:20:00"/>
    <activity type="escort" start_time="18:11:05" end_time="18:22:37" x="-1.5632552968608024" y="53.791411442268775"/>
    <leg mode="car" trav_time="00:25:00"/>
    <activity type="home" start_time="18:47:37" end_time="24:00:00" x="-1.3876467539043311" y="53.932630464963154"/>
  </plan>
</person>

<person id="18">
  <attributes>
    <attribute class="java.lang.String" name="hzone">E00058996</attribute>
    <attribute class="java.lang.Integer" name="age_years">16</attribute>
    <attribute class="java.lang.Integer" name="hid">7</attribute>
  </attributes>
  <plan selected="yes">
    <activity type="home" start_time="00:00:00" end_time="06:28:37" x="-1.406723232210051" y="53.92058549090443"/>
    <leg mode="car" trav_time="00:15:00"/>
    <activity type="work" start_time="06:43:37" end_time="18:28:06" x="-1.5264036491984252" y="53.85154477436126"/>
    <leg mode="car" trav_time="00:15:00"/>
    <activity type="home" start_time="18:43:06" end_time="24:00:00" x="-1.406723232210051" y="53.92058549090443"/>
  </plan>
</person>
<!-- ====================================================================== -->

</population>
```